In [1]:
# LT 29/08/2022
# gnomAD v4 prototyping test using a small numbers of partitions

import hail as hl
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'gnomad',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })

# gnomAD v4 exomes VDS
big_vds = hl.vds.read_vds('gs://gnomad/v4.0/raw/exomes/gnomad_v4.0.vds')

# reduce to one single partition for testing
#small_vds = hl.vds.VariantDataset(
#    big_vds.reference_data._filter_partitions(range(1)),
#    big_vds.variant_data._filter_partitions(range(1)),
#)

# densify as a MatrixTable
mt = hl.vds.to_dense_mt(small_vds)

# remove REF/REF rows left after densification
mt = mt.filter_rows(hl.len(mt.alleles) < 2, keep=False)

# get alleles frequencies
mt = mt.transmute_entries(GT=hl.experimental.lgt_to_gt(mt.LGT, mt.LA))
mt = hl.variant_qc(mt)

# keep only the rows
# persist to avoid double evaluation later 
ht = mt.rows().persist()

# safe guard, before writting to an Australian bucket. Limit to 1 000 000 variants
# note that there is no need to write to Australia, since this is an intermediary result
# it would be preferable to have a bucket hosted in the USA
#if (ht.count() < 1e6) :
    # use cpg utils to find proper path (bucket will change with access level)
    ht.write('big.ht', overwrite=True)


2022-08-29 03:39:29 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2022-08-29 03:39:30 WARN  Utils:69 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


Running on Apache Spark version 3.1.3
SparkUI available at http://4010d8f3bc00:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /home/jupyter/hail-20220829-0339-0.2.97-937922d7f46c.log
2022-08-29 04:08:00 Hail: INFO: wrote table with 7902 rows in 1 partition to small.ht


In [5]:
hl.stop()

In [6]:
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'gnomad',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })
ht = hl.read_table('small.ht')

2022-08-29 04:15:36 Hail: INFO: SparkUI: http://4010d8f3bc00:4041
Running on Apache Spark version 3.1.3
SparkUI available at http://4010d8f3bc00:4041
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /home/jupyter/hail-20220829-0415-0.2.97-937922d7f46c.log


In [7]:
ht.show()

+---------------+----------------+------+--------------------------+
| locus         | alleles        | rsid | variant_qc.dp_stats.mean |
+---------------+----------------+------+--------------------------+
| locus<GRCh38> | array<str>     | str  |                  float64 |
+---------------+----------------+------+--------------------------+
| chr1:11844    | ["G","T"]      | NA   |                 2.62e-01 |
| chr1:11933    | ["AGCC","A"]   | NA   |                 2.62e-01 |
| chr1:11937    | ["C","T"]      | NA   |                 2.62e-01 |
| chr1:11939    | ["G","C"]      | NA   |                 2.62e-01 |
| chr1:11942    | ["T","A"]      | NA   |                 2.62e-01 |
| chr1:11994    | ["T","C"]      | NA   |                 2.63e-01 |
| chr1:12001    | ["C","G"]      | NA   |                 2.63e-01 |
| chr1:12016    | ["G","A"]      | NA   |                 2.63e-01 |
| chr1:12060    | ["CTGGAG","C"] | NA   |                 2.63e-01 |
| chr1:12071    | ["T","A"]      | NA   |                 2.63e-01 |
+---------------+----------------+------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.dp_stats.stdev | variant_qc.dp_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
|                  1.03e+00 |                0.00e+00 |
+---------------------------+-------------------------+

+-------------------------+--------------------------+
| variant_qc.dp_stats.max | variant_qc.gq_stats.mean |
+-------------------------+--------------------------+
|                 float64 |                  float64 |
+-------------------------+--------------------------+
|                8.00e+00 |                 1.20e-05 |
|                8.00e+00 |                 5.99e-06 |
|                8.00e+00 |                 5.99e-06 |
|                8.00e+00 |                 5.99e-06 |
|                8.00e+00 |                 5.99e-06 |
|                8.00e+00 |                 5.19e-05 |
|                8.00e+00 |                 5.19e-05 |
|                8.00e+00 |                 1.72e-04 |
|                1.30e+01 |                 9.19e-04 |
|                1.50e+01 |                 1.10e-03 |
+-------------------------+--------------------------+

+---------------------------+-------------------------+
| variant_qc.gq_stats.stdev | variant_qc.gq_stats.min |
+---------------------------+-------------------------+
|                   float64 |                 float64 |
+---------------------------+-------------------------+
|                  8.48e-03 |                0.00e+00 |
|                  4.24e-03 |                0.00e+00 |
|                  4.24e-03 |                0.00e+00 |
|                  4.24e-03 |                0.00e+00 |
|                  4.24e-03 |                0.00e+00 |
|                  2.95e-02 |                0.00e+00 |
|                  2.89e-02 |                0.00e+00 |
|                  5.60e-02 |                0.00e+00 |
|                  1.56e-01 |                0.00e+00 |
|                  1.55e-01 |                0.00e+00 |
+---------------------------+-------------------------+

+-------------------------+---------------+---------------------+
| variant_qc.gq_stats.max | variant_qc.AC | variant_qc.AF       |
+-------------------------+---------------+---------------------+
|                 floa

In [ ]:
# LT 01/09/2022
# create a gnomAD v4 small test vds with 0.5% partitions

import hail as hl
hl.init(default_reference='GRCh38',
        spark_conf = {
            'spark.hadoop.fs.gs.requester.pays.mode': 'CUSTOM',
            'spark.hadoop.fs.gs.requester.pays.buckets': 'gnomad',
            'spark.hadoop.fs.gs.requester.pays.project.id': 'constraint-232598',
        })

# gnomAD v4 exomes VDS
big_vds = hl.vds.read_vds('gs://gnomad/v4.0/raw/exomes/gnomad_v4.0.vds')

# reduce to one single partition for testing
small_vds = hl.vds.VariantDataset(
    big_vds.reference_data._filter_partitions(range(250)),
    big_vds.variant_data._filter_partitions(range(250)),
)
small_vds.write('gs://cpg-constraint-test/gnomad_v4.0_test.vds')

2022-09-01 03:07:42 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://97d3b1c3146d:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.97-937922d7f46c
LOGGING: writing to /home/jupyter/hail-20220901-0307-0.2.97-937922d7f46c.log


In [2]:
hl.stop()